# Calculate radiance spectrum at the top of the atmosphere

In [ ]:
# Download the CKD datasets if they are here yet
!eradiate data install panellus
!eradiate data install mycena

In [ ]:
import eradiate
import matplotlib.pyplot as plt
import numpy as np

from eradiate import unit_registry as ureg
from eradiate.scenes.atmosphere import MolecularAtmosphere
from eradiate import scenes as ertsc

In [ ]:
eradiate.fresolver.append("../../data")
eradiate.plot.set_style()
plt.style.use("../eradiate.mplstyle")

### Define molecular absorption parameterization
 
We set the simulation mode to "ckd_double", which means that we use a correlated-k distribution and run the simuilation in double precision. 

We select the US standard atmosphere and choose the absorption parameterization "panellus" for hyperspectral simulations at a spectral resolution of 1nm.

In [ ]:
eradiate.set_mode("ckd_double")

atmosphere_gas = MolecularAtmosphere(
    thermoprops={
        "identifier": "afgl_1986-us_standard",
        "z": np.linspace(0.0, 120.0, 121) * ureg.km,
    },
    absorption_data="panellus",  # 'mycena' for 10 nm resolution, 'panellus' for 1 nm
)

### Define an aerosol layer

Here we define a uniform aerosol layer between 0 and 3 km altitude with an optical thickness of 2 at 550 nm. The aerosol optical properties correspond to the continental aerosol distribution as parameterized in the SIX6V RT model. 

In [ ]:
particle_layer = ertsc.atmosphere.ParticleLayer(
    tau_ref=2,
    w_ref=550,
    bottom=0,
    top=3 * ureg.km,
    distribution="uniform",
    dataset="sixsv-continental",
)

Now we combine the molecular atmosphere and the aerosol layer.

In [ ]:
def make_atmosphere(aerosols):
    if aerosols:
        particle_layers = [particle_layer]
    else:
        particle_layers = None

    return ertsc.atmosphere.HeterogeneousAtmosphere(
        molecular_atmosphere=atmosphere_gas,
        particle_layers=particle_layers,
    )

### Define the surface albedo

We load a spectral surface albedo file extracted from HAMSTER data as an ascii file and define an InterpolatedSpectrum to be used in the simulation.

In [ ]:
# spectral albedo data extracted from HAMSTER dataset, read as ascii file
from eradiate.scenes.spectra import InterpolatedSpectrum

albedo_data = np.loadtxt(
    eradiate.fresolver.resolve("HAMSTER_spectral_albedo_Gobabeb_015.txt"),
    skiprows=1,
)
albedo_spectrum = InterpolatedSpectrum(
    wavelengths=albedo_data[:, 0], values=albedo_data[:, 1]
)

## Run the computation

We now compute the reflected radiance spectrum for two cases: first, only with the molecular component of the atmosphere, then with the aerosol layer. For convenience, we write a small function to add a bit of automation to the simulation setup.

In [ ]:
# define the atmosphere experiment function
def run_exp(spp: int, wmin, wmax, atmosphere, vza, vaa, sza, saa, integrator=None):
    """
    Generate and run an atmosphere experiment.
    """

    if integrator is None:
        integrator = ertsc.integrators.PiecewiseVolPathIntegrator()

    # set moment to True to compute the variance of the radiance
    integrator.moment = True

    measure = ertsc.measure.MultiDistantMeasure.hplane(
        zeniths=np.array(vza),
        azimuth=vaa,
        spp=spp,
        srf={"type": "uniform", "wmin": wmin, "wmax": wmax},
    )

    exp = eradiate.experiments.AtmosphereExperiment(
        surface={
            "type": "lambertian",
            "reflectance": albedo_spectrum,
        },
        illumination={
            "type": "directional",
            "zenith": sza,
            "azimuth": saa,
        },
        atmosphere=atmosphere,
        measures=measure,
        integrator=integrator,
    )

    return eradiate.run(exp, spp=spp)

In [ ]:
# first simulation with no aerosols
results_molecular = run_exp(
    spp=100,
    wmin=600,
    wmax=850,
    atmosphere=make_atmosphere(aerosols=False),
    vza=60,
    vaa=75,
    sza=30,
    saa=160,
    integrator=None,
)

# second simulation with aerosols
results_aerosols = run_exp(
    spp=100,
    wmin=600,
    wmax=850,
    atmosphere=make_atmosphere(aerosols=True),
    vza=60,
    vaa=75,
    sza=30,
    saa=160,
    integrator=None,
)

### Plot spectra

First, plot reflected radiance spectra at TOA:

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# use xarray to select data and squeeze them to remove unwanted array dimensions
# thanks to this, no need to select on dimensions of size 1
ds_molecular = results_molecular[["radiance", "radiance_var"]].squeeze()
ds_aerosols = results_aerosols[["radiance", "radiance_var"]].squeeze()
irradiance = results_molecular["irradiance"].squeeze()

# now, make the plot
fig, axs = plt.subplots(
    2, 1, height_ratios=[0.5, 1], figsize=(6, 4.5), sharex=True, dpi=120
)

# add solar irradiance for comparison
ax = axs[0]
ax.plot(
    irradiance.w.values,
    irradiance.values,
    color="C2",
    label="Solar horizontal irradiance",
    lw=1,
)
ax.legend(fontsize="small")
ax.set_ylabel("Irradiance\n[W m$^{-2}$ nm$^{-1}$]")

# add TOA reflected radiance
ax = axs[1]
for ds, color, label in [
    (ds_molecular, "C0", "Molecular"),
    (ds_aerosols, "C1", "Aerosols"),
]:
    wavelengths = ds["w"].values
    brf = ds["radiance"].values
    brf_var = ds["radiance_var"].values
    std = np.sqrt(brf_var)
    upper = brf + 2.0 * std
    lower = brf - 2.0 * std

    # plot BRF and Monte Carlo uncertainty
    ax.plot(wavelengths, brf, label=label, color=color)
    ax.fill_between(wavelengths, upper, lower, alpha=0.25, color=color)

ax.set_xlabel("Wavelength [nm]")
ax.set_ylabel("Radiance\n[W m$^{-2}$ sr$^{-1}$ nm$^{-1}$]")
ax.set_xlim(wavelengths[0], wavelengths[-1])

handles, labels = ax.get_legend_handles_labels()
handles.append(mpatches.Patch(color="black", alpha=0.25, label="2σ uncertainty"))
ax.legend(handles=handles, fontsize="small")

plt.savefig("hyperspectral_radiance_toa.pdf", bbox_inches="tight")
plt.show()
plt.close()

The result shows the radiance spectrum. We used 100 samples only, this means we can expect significant noise (as visualized by two standard deviations). We clearly see the O2A (\~760 nm) and O2B (\~686nm) bands and the water vapor absorption bands around 725nm and 825nm.

Next, we can plot the TOA BRF and compare with the surface albedo from HAMSTER:

In [ ]:
# now, plot the TOA BRF

# first, compute BRF variance
def add_brf_var(ds, return_modified=False):
    x = np.pi / ds["irradiance"]
    ds["brf_var"] = ds["radiance_var"] * x**2

    if return_modified:
        return ds


add_brf_var(results_molecular)
add_brf_var(results_aerosols)

# then, select the data we are going to plot
ds_molecular = results_molecular[["brf", "brf_var"]].squeeze()
ds_aerosols = results_aerosols[["brf", "brf_var"]].squeeze()

# now, plot the data as we did for radiance
fig, ax = plt.subplots(figsize=(6, 3), dpi=120)

for ds, color, label in [
    (ds_molecular, "C0", "Molecular"),
    (ds_aerosols, "C1", "Aerosols"),
]:
    wavelengths = ds["w"].values
    brf = ds["brf"].values
    brf_var = ds["brf_var"].values
    std = np.sqrt(brf_var)
    upper = brf + 2.0 * std
    lower = brf - 2.0 * std

    # plot BRF and Monte Carlo uncertainty
    ax.plot(wavelengths, brf, label=label, color=color)
    ax.fill_between(wavelengths, upper, lower, alpha=0.25, color=color)

# add the surface albedo for comparison
ax.plot(
    albedo_spectrum.wavelengths.m_as("nm"),
    albedo_spectrum.values,
    color="C2",
    label="Surface albedo",
    lw=1,
)

ax.set_xlabel("Wavelength [nm]")
ax.set_ylabel("Reflectance [−]")
ax.set_xlim(wavelengths[0], wavelengths[-1])

handles, labels = ax.get_legend_handles_labels()
handles.append(mpatches.Patch(color="black", alpha=0.25, lw=1, label="2σ uncertainty"))
ax.legend(handles=handles, fontsize="small")

plt.savefig("hyperspectral_brf_toa.pdf", bbox_inches="tight")
plt.show()
plt.close()